<a href="https://colab.research.google.com/github/uoacapstonegroup6/CapstoneUOATeam6/blob/main/Next_steps_fineTuningMedViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Omid-Nejati/MedViT.git

In [ ]:
cd /content/MedViT

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary

In [ ]:
!pip install timm
!pip install einops

In [ ]:
from MedViT import MedViT_small as tiny

In [ ]:
model = tiny()

In [ ]:
model.proj_head[0]

In [ ]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=2, bias=True) #out_features=3 for Fracture 3D Mnist

In [ ]:
model = model.cuda()

In [ ]:
# Run the Dataset processing code (already in another code)

# Fine-tuning

In [ ]:
# Freeze all layers except the last 5
for name, param in model.named_parameters():
    param.requires_grad = False  # Freeze all layers by default

# Unfreeze the last 5 layers
for name, param in list(model.named_parameters())[-5:]:
    param.requires_grad = True

# Define the loss function
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

# Define the optimizer only for the trainable parameters (last 5 layers)
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9)

# Train
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]' % (epoch + 1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        # Forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
